# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [23]:
import pandas
import numpy as np
from azureml.core import Workspace, Dataset, Experiment, ScriptRunConfig
from azureml.core.environment import Environment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import joblib
import os
import shutil
import logging
from azureml.widgets import RunDetails
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
import requests

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [27]:


subscription_id = '9b72f9e6-56c5-4c16-991b-19c652994860'
resource_group = 'aml-quickstarts-164607'
workspace_name = 'quick-starts-ws-164607'



ws = Workspace(subscription_id, resource_group, workspace_name)

experiment_name = 'hyptuneth'

exp=Experiment(ws, experiment_name)

env=Environment(name='myenv')

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-164607
Azure region: southcentralus
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-164607


In [11]:
cluster_name = "AIcompinst3"



try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
{'currentNodeCount': 4, 'targetNodeCount': 4, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 4, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-11-28T22:21:08.144000+00:00', 'errors': None, 'creationTime': '2021-11-28T22:17:39.661194+00:00', 'modifiedTime': '2021-11-28T22:17:55.277090+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [12]:


subscription_id = '9b72f9e6-56c5-4c16-991b-19c652994860'
resource_group = 'aml-quickstarts-164607'
workspace_name = 'quick-starts-ws-164607'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='ethdata')
dataset.to_pandas_dataframe()

,Column1,Open,High,Low,Close,Volume,Dividends,Stock Splits,row_num,row_weight
0,2021-09-01 00:00:00,3413.356201,3444.100586,3397.403076,3401.603027,0,0,0,0,1.0
1,2021-09-01 01:00:00,3401.612061,3436.218506,3387.408447,3427.627441,315990016,0,0,1,1.0
2,2021-09-01 02:00:00,3421.749512,3444.784668,3417.847412,3423.020020,255442944,0,0,2,1.0
3,2021-09-01 03:00:00,3417.972412,3443.150146,3412.599854,3443.150146,115654656,0,0,3,1.0
4,2021-09-01 04:00:00,3440.952637,3460.789062,3433.136719,3457.959961,292694016,0,0,4,1.0
...,...,...,...,...,...,...,...,...,...,...
1306,2021-10-27 18:00:00,3990.167969,4007.307617,3980.632324,3983.689697,0,0,0,1306,1.0
1307,2021-10-27 19:00:00,3972.544189,4005.208740,3948.058838,4004.789551,647258112,0,0,1307,1.0
1308,2021-10-27 20:00:00,4003.598877,4024.692139,3965.883789,3984.781738,0,0,0,1308,1.0
1309,2021-10-27 21:00:00,3986.961914,4011.909424,3978.768311,3999.894775,25716736,0,0,1309,1.0


In [13]:
"""if "training" not in os.listdir():
    os.mkdir("./training")
    
shutil.copy(os.getcwd()+'/train.py',os.getcwd()+'/training/train.py')"""

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook164607/code/Users/odl_user_164607/training/train.py'

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [44]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_amount = 0.2,
                      evaluation_interval=1,
                      delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        # There's only one parameter, so grid sampling will try each value - with multiple parameters it would try every combination
        '--epsilon': uniform( 1e-8,1.0),
        '--max_iter': choice(range(50,1000,25))
    }
)

#TODO: Create your estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory='.',
              script='train.py',
              arguments=['--input_data',dataset.as_named_input('ethdata')],              
              compute_target = compute_target,
              environment=env)
              

hyperdrive_run_config = HyperDriveConfig(
                                      hyperparameter_sampling=param_sampling, 
                                      policy=early_termination_policy,
                                      run_config=estimator,  
                                      primary_metric_name='RMSE', 
                                      primary_metric_goal=PrimaryMetricGoal.MINIMIZE, 
                                      max_total_runs=25,
                                      max_concurrent_runs=4)

In [45]:
#TODO: Submit your experiment
hyprun = exp.submit(hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [46]:
RunDetails(hyprun).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [21]:
#TODO: Save the best model

model = hyprun.get_best_run_by_primary_metric()
model = model.register_model(model_name='hyptun_eth', model_path='outputs/hyptun_model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "aciservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:

# Load image data
data = [1.25, 1.5]
# Post raw data to scoring URI
res = request.post(url=service[0].scoring_uri, data=data, headers={'R7': np.float()})

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.scoring_uri)
print(service.swagger_uri)
service.delete()